# MLOps

MLOps is a set of standardized processes and technology capabilities for building, deploying, and operationalizing ML systems rapidly and reliably.

## The MLOps Lifecycle

The MLOps lifecycle encompasses seven processes, as shown below:

<img src="../img/mlops.png" alt="MLOPs Lifecycle" style="width: 500px;"/>

The processes can consist of the following:

- **ML development** concerns experimenting and developing a robust and reproducible model training procedure (training pipeline code), which consists of multiple tasks from data preparation and transformation to model training and evaluation.
<br><br>
- **Training operationalization** concerns automating the process of packaging, testing, and deploying repeatable and reliable training pipelines.
<br><br>
- **Continuous training** concerns repeatedly executing the training pipeline in response to new data or to code changes, or on a schedule, potentially with new training settings.
<br><br>
- **Model deployment** concerns packaging, testing, and deploying a model to a serving environment for online experimentation and production serving.
<br><br>
- **Prediction serving** is about serving the model that is deployed in production for inference.
<br><br>
- **Continuous monitoring** is about monitoring the effectiveness and efficiency of a deployed model.
<br><br>
- **Data and model management** is a central, cross-cutting function for governing ML artifacts to support auditability, traceability, and compliance. Data and model management can also promote shareability, reusability, and discoverability of ML assets.

Source: https://cloud.google.com/resources/mlops-whitepaper

## Google Cloud Platform ML Toolchain

Google's Cloud Platform (GCP) offers a rich set of services to help one accomplish MLOps. In this Notebook, I will walk you through a number of different components available in GCP.

### AI Platform

<img src="../img/GCP_AIP.png" alt="MLOPs Lifecycle" style="width: 90px; float: left;"/>

Google AI Platform can be used to train your machine learning models at scale, to host your trained model in the cloud, and to use your model to make predictions about new data. The AI platform supports all of the steps in the following workflow:

<img src="../img/GCP_ML_Workflow.png" alt="MLOPs Lifecycle" style="width:700px; float: left;"/>

#### Training Service

The AI Platform training service allows you to train models using a wide range of different customization options.You can select many different machine types to power your training jobs, enable distributed training, use hyperparameter tuning, and accelerate with GPUs and TPUs.

You can also select different ways to customize your training application. You can submit your input data for AI Platform to train using a built-in algorithm (beta). If the built-in algorithms do not fit your use case, you can submit your own training application to run on AI Platform, or build a custom container with your training application and its dependencies to run on AI Platform.

#### Prediction Service

The AI Platform prediction service allows you to serve predictions based on a trained model, whether or not the model was trained on AI Platform.

#### Data Labeling Service

AI Platform Data Labeling Service (beta) lets you request human labeling for a dataset that you plan to use to train a custom machine learning model. You can submit a request to label your video, image, or text data.

To submit a labeling request, you provide a representative sample of labeled data, specify all the possible labels for your dataset, and provide some instructions for how to apply those labels. The human labelers follow your instructions, and when the labeling request is complete, you get your annotated dataset that you can use to train a machine learning model.

Learn more about Google's AI Platform through their [Introduction to AI Platform](https://cloud.google.com/ai-platform/docs/technical-overview).

### Google BigQuery

<img src="../img/GCP_BigQuery.png" alt="MLOPs Lifecycle" style="width: 200px; float: left;"/>

BigQuery is a fully-managed enterprise data warehouse that helps you manage and analyze your data with built-in features like machine learning, geospatial analysis, and business intelligence. BigQuery's serverless architecture lets you use SQL queries to answer your organization's biggest questions with zero infrastructure management. BigQuery's scalable, distributed analysis engine lets you query terabytes in seconds and petabytes in minutes.

Learn more via [What is BigQuery?](https://cloud.google.com/bigquery/docs/introduction)

### Google Kubernetes Engine (GKE)

<img src="../img/GCP_GKE.png" alt="MLOPs Lifecycle" style="width: 300px; float: left;"/>

Google Kubernetes Engine (GKE) provides a managed environment for deploying, managing, and scaling your containerized applications using Google infrastructure. The GKE environment consists of multiple machines (specifically, Compute Engine instances) grouped together to form a cluster.

Learn more via Google's [GKE Overview](https://cloud.google.com/kubernetes-engine/docs/concepts/kubernetes-engine-overview).

### Google Cloud Build

<img src="../img/GCP_Build.png" alt="MLOPs Lifecycle" style="width: 80px; float: left;"/>

Cloud Build is a service that executes your builds on Google Cloud Platform's infrastructure. Cloud Build can import source code from a variety of repositories or cloud storage spaces, execute a build to your specifications, and produce artifacts such as Docker containers or Java archives.

Learn more via Google's [Overview of Cloud Build](https://cloud.google.com/build/docs/overview).

## Kubeflow

<img src="../img/Kubeflow.png" alt="MLOPs Lifecycle" style="width: 120px; float: left;"/>

Kubeflow is an end-to-end Machine Learning (ML) platform for Kubernetes, it provides components for each stage in the ML lifecycle, from exploration through to training and deployment. Operators can choose what is best for their users, there is no requirement to deploy every component. 

Learn more via the [Kubeflow Overview page](https://www.kubeflow.org/docs/started/kubeflow-overview/).

## MLOps on GCP

By combining all of the above components we can create an MLOps environment in GCP. See the diagram below for the big picture.

<img src="../img/GCP_MLOps.png" alt="MLOPs Lifecycle" style="float: left;"/>

This environment supports a number of different user workflows. Our primary goal, continuous training and deployment of ML models, can be done in a fully autonomous fashion with this environment. Second, through KubeFlow's web-based GUI, an operator can manually kick off pipelines. Finally, we can leverage AI Platform Notebooks to support prototyping and building out pipelines.

### Continuous Training

The continuous training workflow is accomplished as follows:

- **Datasets** are stored either directly in a Git repository on GitHub (or other similar provider), in Google Cloud Storage, or in the AI Platform datasets repository. 
<br><br>
- **Training algorithms** are stored in a Git repository on GitHub (or other similar provider).
<br><br>
- **KubeFlow pipeline definitions** are also stored in your Git repository. Dockerfiles for generating pipeline container images are also stored in the Git repository.
<br><br>
- **Google Cloud Build** is integrated with your Git repository. Any opened pull request or push to the Git repository will trigger the Google Cloud Build. The Cloud Build pipeline definition is also stored in your Git repository.

When the Cloud Build is triggered:

- **Pipeline Container Images** are generated by the cloud build pipeline. This includes a **trainer image** (the image used to run the training algorithms), and a **baseline image** (the image used to run the KubeFlow pipeline steps). 
<br><br>
- **A KubeFlow Pipeline** is generated for this build. This means that if you modify the KubeFlow pipeline definition (or any of the pipeline container images), the definitions in Git will be used. The end result is that your pipeline is dynamically generated from your source code.
<br><br>
- **The KubeFlow Pipeline** is executed. The following steps are generally executed (although this is fully configurable via your pipeline definition):
<br><br>
  - **Step: Data Storage/Processing** is responsible for preparing the datasets (training vs. test vs. validation). Recall that your dataset may be stored in multiple locations. This step in your KubeFlow pipeline will need to be defined to pull the dataset from the correct location.
<br><br>
  - **Step: Hyperparameter Tuning** runs your training algorithm with a variety of different hyperparameters in order to search for the best hyperparameters for the training task. This uses Google AI Platform Vizier for training in parallel using serverless training jobs.
<br><br>
  - **Step: Serverless Training** is used to train the final release candidate model with the best hyperparameters found in the previous step.
<br><br>
  - **Step: Scalable Inference** is done to run model prediction on an evaluation dataset. This step may include a conditional step to halt the pipeline if the model accuracy/precision/recall is below a pre-specified threshold.
<br><br>
  - **Step: Model Promotion** is responsible for promoting our release candidate model to production if the model meets the necessary criteria. Promoted models are versioned and stored using AI Platform Models. Once the model is promoted, it is immediately used in production.

### Example Cloud Build Execution

The following was kicked off after a commit was made to our Git repository:
<br>
<img src="../img/GCP_CloudBuild_Ex.png" alt="MLOPs Lifecycle" style="width: 600px; float: left;"/>

### Example KubeFlow Pipeline Execution

The following KubeFlow pipeline was generated and executed by our Cloud Build execution:
<br>
<img src="../img/GCP_KubeFlow_Ex.png" alt="MLOPs Lifecycle" style="width: 400px; float: left;"/>

### Example Promoted Model

The following model was automatically created and promoted by our KubeFlow pipeline:
<br>
<img src="../img/GCP_Models_Ex.png" alt="MLOPs Lifecycle" style="width: 670px; float: left;"/>

### Manual Execution

While our primary goal is autonomous continuous training, KubeFlow also supports manual pipeline executions:
<br>
<img src="../img/KubeFlow_Manual.png" alt="MLOPs Lifecycle" style="width: 470px; float: left;"/>

### Prototyping with AI Platform Notebooks

When getting started with MLOps, it is essential to deploy a Jupyter Lab server in your GCP environment. This server will allow you to create notebooks that can deploy and run ML pipelines. In essence, this is useful for rapid prototyping and iterative pipeline development. 
<br>
<img src="../img/GCP_AI_Notebooks.png" alt="MLOPs Lifecycle" style="width: 670px; float: left;"/>

## Take Home Challenge: Deploying an MLOps Environment

Google provides several resources to help you get started:

* First, [create an AI Platform Notebooks instance](https://github.com/GoogleCloudPlatform/mlops-on-gcp/blob/master/examples/mlops-env-on-gcp/creating-notebook-instance/README.md).
* Next, [deploy your KubeFlow pipeline infrastructure](https://github.com/GoogleCloudPlatform/mlops-on-gcp/blob/master/examples/mlops-env-on-gcp/provisioning-kfp/README.md).
* Finally, follow the [Continuous training with scikit-learn and Cloud AI Platform workshop](https://github.com/GoogleCloudPlatform/mlops-on-gcp/tree/master/workshops/kfp-caip-sklearn). This workshop includes three labs that take you from zero to getting everything we have gone through in this notebook up and running.